<a href="https://colab.research.google.com/github/Polapob/Obama-Text-Generation/blob/main/Obama_text_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Text generation with an RNN

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/text/text_generation"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/text/text_generation.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/text/text_generation.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/text/text_generation.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

While some of the sentences are grammatical, most do not make sense. The model has not learned the meaning of words, but consider:

* The model is character-based. When training started, the model did not know how to spell an English word, or that words were even a unit of text.

* The structure of the output resembles a play—blocks of text generally begin with a speaker name, in all capital letters similar to the dataset.

* As demonstrated below, the model is trained on small batches of text (100 characters each), and is still able to generate a longer sequence of text with coherent structure.

## Setup

### Import TensorFlow and other libraries

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing

import numpy as np
import os
import time

### Download the Obama dataset


In [ ]:
!wget https://raw.githubusercontent.com/samim23/obama-rnn/master/input.txt

--2021-01-11 03:55:02--  https://raw.githubusercontent.com/samim23/obama-rnn/master/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4250014 (4.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   4.05M  --.-KB/s    in 0.05s   

2021-01-11 03:55:02 (87.1 MB/s) - ‘input.txt’ saved [4250014/4250014]



In [ ]:
path_to_file = "input.txt"

### Read the data

First, look in the text:

In [ ]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print('Length of text: {} characters'.format(len(text)))

Length of text: 4224143 characters


In [ ]:
# Take a look at the first 250 characters in text
print(text[:250])

To Chip, Kathy, and Nancy, who graciously shared your father with a nation that loved him; to Walter's friends, colleagues, protégés, and all who considered him a hero; to the men of the Intrepid; to all of you who are gathered here today; I am honor


In [ ]:
# The unique characters in the file
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

121 unique characters


## Process the text

### Vectorize the text

Before training, you need to convert the strings to a numerical representation. 

The `preprocessing.StringLookup` layer can convert each character into a numeric ID. It just needs the text to be split into tokens first.

In [ ]:
chars = tf.strings.unicode_split(text[:50],input_encoding="UTF-8")
chars

<tf.Tensor: shape=(50,), dtype=string, numpy=
array([b'T', b'o', b' ', b'C', b'h', b'i', b'p', b',', b' ', b'K', b'a',
       b't', b'h', b'y', b',', b' ', b'a', b'n', b'd', b' ', b'N', b'a',
       b'n', b'c', b'y', b',', b' ', b'w', b'h', b'o', b' ', b'g', b'r',
       b'a', b'c', b'i', b'o', b'u', b's', b'l', b'y', b' ', b's', b'h',
       b'a', b'r', b'e', b'd', b' ', b'y'], dtype=object)>

Now create the `preprocessing.StringLookup` layer:

In [ ]:
ids_from_chars = preprocessing.StringLookup(
    vocabulary=list(vocab))

In [ ]:
vocab2 = ids_from_chars.get_vocabulary()
for i in vocab2:
  print(i)

It converts form tokens to character IDs, padding with `0`:

In [ ]:
ids = ids_from_chars(chars)
ids

<tf.Tensor: shape=(50,), dtype=int64, numpy=
array([52, 76,  3, 35, 69, 70, 77, 14,  3, 43, 62, 81, 69, 86, 14,  3, 62,
       75, 65,  3, 46, 62, 75, 64, 86, 14,  3, 84, 69, 76,  3, 68, 79, 62,
       64, 70, 76, 82, 80, 73, 86,  3, 80, 69, 62, 79, 66, 65,  3, 86])>

Since the goal of this tutorial is to generate text, it will also be important to invert this representation and recover human-readable strings from it. For this you can use `preprocessing.StringLookup(..., invert=True)`.  

Note: Here instead of passing the original vocabulary generated with `sorted(set(text))` use the `get_vocabulary()` method of the `preprocessing.StringLookup` layer so that the padding and `[UNK]` tokens are set the same way.

In [ ]:
chars_from_ids = tf.keras.layers.experimental.preprocessing.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True)

This layer recovers the characters from the vectors of IDs, and returns them as a `tf.RaggedTensor` of characters:

In [ ]:
chars = chars_from_ids(ids)
chars

<tf.Tensor: shape=(50,), dtype=string, numpy=
array([b'T', b'o', b' ', b'C', b'h', b'i', b'p', b',', b' ', b'K', b'a',
       b't', b'h', b'y', b',', b' ', b'a', b'n', b'd', b' ', b'N', b'a',
       b'n', b'c', b'y', b',', b' ', b'w', b'h', b'o', b' ', b'g', b'r',
       b'a', b'c', b'i', b'o', b'u', b's', b'l', b'y', b' ', b's', b'h',
       b'a', b'r', b'e', b'd', b' ', b'y'], dtype=object)>

You can `tf.strings.reduce_join` to join the characters back into strings. 

In [ ]:
tf.strings.reduce_join(chars, axis=-1).numpy()

b'To Chip, Kathy, and Nancy, who graciously shared y'

In [ ]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

### The prediction task

Given a character, or a sequence of characters, what is the most probable next character? This is the task you're training the model to perform. The input to the model will be a sequence of characters, and you train the model to predict the output—the following character at each time step.

Since RNNs maintain an internal state that depends on the previously seen elements, given all the characters computed until this moment, what is the next character?


### Create training examples and targets

Next divide the text into example sequences. Each input sequence will contain `seq_length` characters from the text.

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.

So break the text into chunks of `seq_length+1`. For example, say `seq_length` is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello".

To do this first use the `tf.data.Dataset.from_tensor_slices` function to convert the text vector into a stream of character indices.

In [ ]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(4224143,), dtype=int64, numpy=array([52, 76,  3, ...,  2,  2,  2])>

In [ ]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

visualize each category

In [ ]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

T
o
 
C
h
i
p
,
 
K


In [ ]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

The `batch` method lets you easily convert these individual characters to sequences of the desired size.

In [ ]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)
count=1
for seq in sequences.take(2):
  print("Seq =",count)
  print()
  print(chars_from_ids(seq))
  print()
  count +=1

Seq = 1

tf.Tensor(
[b'T' b'o' b' ' b'C' b'h' b'i' b'p' b',' b' ' b'K' b'a' b't' b'h' b'y'
 b',' b' ' b'a' b'n' b'd' b' ' b'N' b'a' b'n' b'c' b'y' b',' b' ' b'w'
 b'h' b'o' b' ' b'g' b'r' b'a' b'c' b'i' b'o' b'u' b's' b'l' b'y' b' '
 b's' b'h' b'a' b'r' b'e' b'd' b' ' b'y' b'o' b'u' b'r' b' ' b'f' b'a'
 b't' b'h' b'e' b'r' b' ' b'w' b'i' b't' b'h' b' ' b'a' b' ' b'n' b'a'
 b't' b'i' b'o' b'n' b' ' b't' b'h' b'a' b't' b' ' b'l' b'o' b'v' b'e'
 b'd' b' ' b'h' b'i' b'm' b';' b' ' b't' b'o' b' ' b'W' b'a' b'l' b't'
 b'e' b'r' b"'"], shape=(101,), dtype=string)

Seq = 2

tf.Tensor(
[b's' b' ' b'f' b'r' b'i' b'e' b'n' b'd' b's' b',' b' ' b'c' b'o' b'l'
 b'l' b'e' b'a' b'g' b'u' b'e' b's' b',' b' ' b'p' b'r' b'o' b't'
 b'\xc3\xa9' b'g' b'\xc3\xa9' b's' b',' b' ' b'a' b'n' b'd' b' ' b'a' b'l'
 b'l' b' ' b'w' b'h' b'o' b' ' b'c' b'o' b'n' b's' b'i' b'd' b'e' b'r'
 b'e' b'd' b' ' b'h' b'i' b'm' b' ' b'a' b' ' b'h' b'e' b'r' b'o' b';'
 b' ' b't' b'o' b' ' b't' b'h' b'e' b' ' b'm' b'e' b'n' b' ' b

It's easier to see what this is doing if you join the tokens back into strings:

In [ ]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b"To Chip, Kathy, and Nancy, who graciously shared your father with a nation that loved him; to Walter'"
b's friends, colleagues, prot\xc3\xa9g\xc3\xa9s, and all who considered him a hero; to the men of the Intrepid; to al'
b'l of you who are gathered here today; I am honored to be here to pay tribute to the life and times of'
b' the man who chronicled our time.\n\nI did not know Mr. Cronkite personally.  And my regret is made mor'
b'e acute by the stories that have been shared here today. Nor, for that matter, did I know him any bet'


For training you'll need a dataset of `(input, label)` pairs. Where `input` and 
`label` are sequences. At each time step the input is the current character and the label is the next character. 

Here's a function that takes a sequence as input, duplicates, and shifts it to align the input and label for each timestep:

In [ ]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [ ]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [ ]:
dataset = sequences.map(split_input_target)

<MapDataset shapes: ((100,), (100,)), types: (tf.int64, tf.int64)>

Visualize some dataset.

In [ ]:
for input_example, target_example in  dataset.take(3):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())
    print()

Input : b'To Chip, Kathy, and Nancy, who graciously shared your father with a nation that loved him; to Walter'
Target: b"o Chip, Kathy, and Nancy, who graciously shared your father with a nation that loved him; to Walter'"

Input : b's friends, colleagues, prot\xc3\xa9g\xc3\xa9s, and all who considered him a hero; to the men of the Intrepid; to a'
Target: b' friends, colleagues, prot\xc3\xa9g\xc3\xa9s, and all who considered him a hero; to the men of the Intrepid; to al'

Input : b'l of you who are gathered here today; I am honored to be here to pay tribute to the life and times o'
Target: b' of you who are gathered here today; I am honored to be here to pay tribute to the life and times of'



### Create training batches

You used `tf.data` to split the text into manageable sequences. But before feeding this data into the model, you need to shuffle the data and pack it into batches.

In [ ]:
# Batch size
BATCH_SIZE = 128

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset shapes: ((128, 100), (128, 100)), types: (tf.int64, tf.int64)>

## Build The Model

This section defines the model as a `keras.Model` subclass 
This model has three layers:

* `tf.keras.layers.Embedding`: The input layer. A trainable lookup table that will map each character-ID to a vector with `embedding_dim` dimensions;
* `tf.keras.layers.GRU`: A type of RNN with size `units=rnn_units` (You can also use an LSTM layer here.)
* `tf.keras.layers.Dense`: The output layer, with `vocab_size` outputs. It outpts one logit for each character in the vocabulary. These are the log-liklihood of each character according to the model.

In [ ]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [ ]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True, 
                                   return_state=True,dropout=0.2)
    """self.gru2 = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True, 
                                   return_state=True,dropout=0.2)
    self.gru3 = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True, 
                                   return_state=True,dropout=0.2)"""
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    #x, states = self.gru2(x, initial_state=states, training=training)
    #x,states = self.gru3(x,initial_state = states,training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else: 
      return x

In [ ]:
model = MyModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

Note: For training you could use a `keras.Sequential` model here. To  generate text later you'll need to manage the RNN's internal state. It's simpler to include the state input and output options upfront, than it is to rearrange the model architecture later. For more details asee the [Keras RNN guide](https://www.tensorflow.org/guide/keras/rnn#rnn_state_reuse).

## Try the model

Now run the model to see that it behaves as expected.

First check the shape of the output:

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(128, 100, 123) # (batch_size, sequence_length, vocab_size)


In the above example the sequence length of the input is `100` but the model can be run on inputs of any length:

In [ ]:
model.summary()

Model: "my_model_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_17 (Embedding)     multiple                  31488     
_________________________________________________________________
gru_18 (GRU)                 multiple                  3938304   
_________________________________________________________________
dense_14 (Dense)             multiple                  126075    
Total params: 4,095,867
Trainable params: 4,095,867
Non-trainable params: 0
_________________________________________________________________


To get actual predictions from the model you need to sample from the output distribution, to get actual character indices. This distribution is defined by the logits over the character vocabulary.

Note: It is important to _sample_ from this distribution as taking the _argmax_ of the distribution can easily get the model stuck in a loop.

Try it for the first example in the batch:

In [ ]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

This gives us, at each timestep, a prediction of the next character index:

In [ ]:
sampled_indices

array([ 53, 117,  54,  47,  43,  74,  54,  63, 108, 115, 101,  38,  69,
        96,  14,  41,  82,  18,  44, 111,  50, 108,  25,  48,  65,  69,
        71,  99, 121, 103,  16, 104,  16, 108,  65,   2,  40, 114,  51,
        54,  66,  88,  45,  92, 118, 116,  37,  60,  97,  74,   3,  17,
        11,  34,  93, 119,  78, 120,  41,  80,   2,  30,  48,  37,  37,
        78, 117, 106,  96,  13,  29,   5, 119,  96,  71, 112,  22,  37,
        98,  77, 102,  11,  50,  48, 115,  46,  96, 109, 104,  70,  39,
         7,   5,  65, 103,  36, 114,  39,   9,  96])

Decode these to see the text predicted by this untrained model:

In [ ]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b' -- let there be no doubt: Islam is a part of America. And I believe that America holds within her t'

Next Char Predictions:
 b'U\xe2\x80\x94VOKmVb\xd6\xb9\xd7\xaa\xc3\xb3Fh\xc3\xa8,Iu0L\xd7\x93R\xd6\xb97Pdhj\xc3\xaf\xe2\x80\x9d\xc4\x85.\xc4\x99.\xd6\xb9d\nH\xd7\xa8SVe\xc2\x92M\xc3\x93\xe2\x80\x98\xe2\x80\x93E]\xc3\xa9m /)B\xc3\xa0\xe2\x80\x99q\xe2\x80\x9cIs\n<PEEq\xe2\x80\x94\xd6\xb7\xc3\xa8+;"\xe2\x80\x99\xc3\xa8j\xd7\x944E\xc3\xadp\xc3\xb4)RP\xd7\xaaN\xc3\xa8\xd6\xbc\xc4\x99iG%"d\xc4\x85D\xd7\xa8G\'\xc3\xa8'


## Train the model

At this point the problem can be treated as a standard classification problem. Given the previous RNN state, and the input this time step, predict the class of the next character.

### Attach an optimizer, and a loss function

The standard `tf.keras.losses.sparse_categorical_crossentropy` loss function works in this case because it is applied across the last dimension of the predictions.

Because your model returns logits, you need to set the `from_logits` flag.


In [ ]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
example_batch_loss = loss(target_example_batch, example_batch_predictions)
mean_loss = example_batch_loss.numpy().mean()
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", mean_loss)

Prediction shape:  (128, 100, 123)  # (batch_size, sequence_length, vocab_size)
Mean loss:         4.8129063


A newly initialized model shouldn't be too sure of itself, the output logits should all have similar magnitudes. To confirm this you can check that the exponential of the mean loss is approximately equal to the vocabulary size. A much higher loss means the model is sure of its wrong answers, and is badly initialized:

In [ ]:
tf.exp(mean_loss).numpy()

123.08883

Configure the training procedure using the `tf.keras.Model.compile` method. Use `tf.keras.optimizers.Adam` with default arguments and the loss function.

In [ ]:
model.compile(optimizer='adam', loss=loss,metrics=["acc"])

### Configure checkpoints

Use a `tf.keras.callbacks.ModelCheckpoint` to ensure that checkpoints are saved during training:

In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints3'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### Execute the training

To keep training time reasonable, use 10 epochs to train the model. In Colab, set the runtime to GPU for faster training.

In [ ]:
EPOCHS = 100

In [ ]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/100
326/326 [==============================] - 37s 108ms/step - loss: 2.9256 - acc: 0.2633
Epoch 2/100
326/326 [==============================] - 36s 108ms/step - loss: 1.6318 - acc: 0.5210
Epoch 3/100
326/326 [==============================] - 36s 108ms/step - loss: 1.2971 - acc: 0.6131
Epoch 4/100
326/326 [==============================] - 37s 110ms/step - loss: 1.1866 - acc: 0.6415
Epoch 5/100
326/326 [==============================] - 36s 109ms/step - loss: 1.1276 - acc: 0.6571
Epoch 6/100
326/326 [==============================] - 37s 110ms/step - loss: 1.0871 - acc: 0.6677
Epoch 7/100
326/326 [==============================] - 36s 109ms/step - loss: 1.0570 - acc: 0.6759
Epoch 8/100
326/326 [==============================] - 36s 108ms/step - loss: 1.0295 - acc: 0.6830
Epoch 9/100
326/326 [==============================] - 36s 108ms/step - loss: 1.0063 - acc: 0.6892
Epoch 10/100
326/326 [==============================] - 37s 109ms/step - loss: 0.9858 - acc: 0.6949
Epoch 11/

KeyboardInterrupt: ignored

## Generate text

The simplest way to generate text with this model is to run it in a loop, and keep track of the model's internal state as you execute it.

![To generate text the model's output is fed back to the input](https://github.com/tensorflow/docs/blob/master/site/en/tutorials/text/images/text_generation_sampling.png?raw=1)

Each time you call the model you pass in some text and an internal state. The model returns a prediction for the next character and its new state. Pass the prediction and state back in to continue generating text.


The following makes a single step prediction:

In [ ]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature=temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "" or "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['','[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices = skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())]) 
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits] 
    predicted_logits, states =  self.model(inputs=input_ids, states=states, 
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "" or "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)
    
    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [ ]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

Run it in a loop to generate some text. Looking at the generated text, you'll see the model knows when to capitalize, make paragraphs and imitates a Shakespeare-like writing vocabulary. With the small number of training epochs, it has not yet learned to form coherent sentences.

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['War'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()

print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)

print(f"\nRun time: {end - start}")

Wars be a shrinking, kid’al expensive As an unity, and that's the break to hangs -- single Summit of the old border.  America wants a reasonable on the need for curbed instead of support that often outlined that force alone will be living it on to the stockpiles.

Nothing say that I worry that the United States has inspired by the new rules that the Department of Defense, the Space Shuttles on Wall Street was tool when I support defenders of the Internet, the Uncondition that we did not know when we support generosity and judgment reform. 

The one nations do not support more than 600 quality, and defeat up their faith and our cars.  In the meantime, we’re going to show that all new insurance seen on Congress over worry that every American still includes some traditional resolved.  A belief in mass shooting, and helping Americans’ greater efforts that will lead direct the Dusad Captain sanctions on Wall Street in courts and resolve as they did the up.  The freedom of excessionary spent

The easiest thing you can do to improve the results is to train it for longer (try `EPOCHS = 30`).

You can also experiment with a different start string, try adding another RNN layer to improve the model's accuracy, or adjust the temperature parameter to generate more or less random predictions.

If you want the model to generate text *faster* the easiest thing you can do is batch the text generation. In the example below the model generates 5 outputs in about the same time it took to generate 1 above. 

In [ ]:
start = time.time()
states = None
next_char = tf.constant(["Charles"])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()

print(result, '\n\n' + '_'*80)


print(f"\nRun time: {end - start}")

tf.Tensor([b"Charlest.\n \nThere are going to be on the office to our veterans?.\n\nPRescurr Goxd Forth, who are someing the lessons of justice and the crisis earlier, the cost of the Bayor and Central Iraqi Set, strikes his part, Fort Concord was gun or dirently, upside shared responsibility in America at -- and we working together in the Israeli people do really, an issue mading their rights because of our lives, and acknowledge that is say, international forums, more complete our destiny illegal issues, sometimes severely pursue at an America that has successfully reducing manifest for its foolists do -- still build -- a promise If America and our adversaries succeeded. And this won't happen your out construction; those days are over.  Bet a basically sign idsna Band in your tax dollars at that time, Colorado \xe2\x80\x93 Assad\xe2\x80\x99s campaign stands; that our common humanity, and hold further devastating climate changes, more affordable for four of our people.  And we finally

## Export the generator

This single-step model can easily be [saved and restored](https://www.tensorflow.org/guide/saved_model), allowing you to use it anywhere a `tf.saved_model` is accepted.

In [ ]:
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')

INFO:tensorflow:Assets written to: one_step/assets


INFO:tensorflow:Assets written to: one_step/assets


In [ ]:
states = None
next_char = tf.constant(["Obama"])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

Obama.  That’s what we need does.

So when Rioseas Highway Assad, Saral,Ery Jack Andost insurance companies, freedo known-goodness.”

So tonight I’d like to talk about hope.  I still find my SEcretary of France to find them -- maybe things like we, the child of Morehouse, Lionwai used a course-coundy half by saying Hoor and more than a cared me then he thought, and the most important issue for our military you can.  And so I want to thank Gir talk to no around the world, I’ve been created networks -- to keep investing in the Senate.  Now is the time for them to reduce the deficit.  But I know some in recent death before us believes that getting some know -- or if the young people look like.  That’s who we made our brothert.

Those still looks alongs if we don’t think two pretucals?  More you lifted building faster lots, only if we don’t give up on them for their own.  And it's way.  I have a hardfronted fire and bear in him, row.  When a child can expand our choices in our girls, and c

## Advanced: Customized Training

The above training procedure is simple, but does not give you much control.
It uses teacher-forcing which prevents bad predictions from being fed back to the model so the model never learns to recover from mistakes.

So now that you've seen how to run the model manually next you'll implement the training loop. This gives a starting point if, for example, you want to implement _curriculum  learning_ to help stabilize the model's open-loop output.

The most important part of a custom training loop is the train step function.

Use `tf.GradientTape` to track the gradients. You can learn more about this approach by reading the [eager execution guide](https://www.tensorflow.org/guide/eager).

The basic procedure is:

1. Execute the model and calculate the loss under a `tf.GradientTape`.
2. Calculate the updates and apply them to the model using the optimizer.

In [ ]:
class CustomTraining(MyModel):
  @tf.function
  def train_step(self, inputs):
      inputs, labels = inputs
      with tf.GradientTape() as tape:
          predictions = self(inputs, training=True)
          loss = self.loss(labels, predictions)
      grads = tape.gradient(loss, model.trainable_variables)
      self.optimizer.apply_gradients(zip(grads, model.trainable_variables))

      return {'loss': loss}

The above implementation of the `train_step` method follows [Keras' `train_step` conventions](https://www.tensorflow.org/guide/keras/customizing_what_happens_in_fit). This is optional, but it allows you to change the behavior of the train step and still use keras' `Model.compile` and `Model.fit` methods.

In [ ]:
model = CustomTraining(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [ ]:
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=["acc"])

In [ ]:
model.fit(dataset, epochs=2)

Epoch 1/2
326/326 [==============================] - 55s 161ms/step - loss: 2.2787
Epoch 2/2
326/326 [==============================] - 53s 160ms/step - loss: 1.4675


Or if you need more control, you can write your own complete custom training loop:

In [ ]:
EPOCHS = 20

mean = tf.metrics.Mean()

for epoch in range(EPOCHS):
    start = time.time()

    mean.reset_states()
    for (batch_n, (inp, target)) in enumerate(dataset):
        logs = model.train_step([inp, target])
        mean.update_state(logs['loss'])

        if batch_n % 50 == 0:
            template = 'Epoch {} Batch {} Loss {}'
            print(template.format(epoch + 1, batch_n, logs['loss']))

    # saving (checkpoint) the model every 5 epochs
    if (epoch + 1) % 5 == 0:
        model.save_weights(checkpoint_prefix.format(epoch=epoch))

    print()
    print('Epoch {} Loss: {:.4f}'.format(epoch + 1, mean.result().numpy()))
    print('Time taken for 1 epoch {} sec'.format(time.time() - start))
    print("_"*80)

model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 0.8680761456489563
Epoch 1 Batch 50 Loss 0.8437008857727051
Epoch 1 Batch 100 Loss 0.8540005683898926
Epoch 1 Batch 150 Loss 0.8422533273696899
Epoch 1 Batch 200 Loss 0.8471494913101196
Epoch 1 Batch 250 Loss 0.8511042594909668
Epoch 1 Batch 300 Loss 0.8330490589141846

Epoch 1 Loss: 0.8456
Time taken for 1 epoch 53.2582061290741 sec
________________________________________________________________________________
Epoch 2 Batch 0 Loss 0.8430423140525818
Epoch 2 Batch 50 Loss 0.8546546101570129
Epoch 2 Batch 100 Loss 0.8338348269462585
Epoch 2 Batch 150 Loss 0.8508053421974182
Epoch 2 Batch 200 Loss 0.8371874094009399
Epoch 2 Batch 250 Loss 0.8394181728363037
Epoch 2 Batch 300 Loss 0.8378104567527771

Epoch 2 Loss: 0.8413
Time taken for 1 epoch 52.41692805290222 sec
________________________________________________________________________________
Epoch 3 Batch 0 Loss 0.8579710125923157
Epoch 3 Batch 50 Loss 0.832732081413269
Epoch 3 Batch 100 Loss 0.8117845058441162
E

In [ ]:
one_step_model_new = OneStep(model, chars_from_ids, ids_from_chars)

In [ ]:
start = time.time()
states = None
next_char = tf.constant(["Book"])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model_new.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()

print(result, '\n\n' + '_'*80)


print(f"\nRun time: {end - start}")

tf.Tensor([b"Bookes; John F. Kennedy\xe2\x80\x99s lesse racists, our nation's history, I said then that their coverage day will shape our obligations with the wealthiest at the victims of your communities and individuals who had come together to action, which I believe deeply in the face of all who wear the same grids of all nations to steep you.  They done sole on Facebook whenever three-years down the road that has known aside where contrarys to those who died in coming together and we can generate further among the Republican senator who are watching where the majority of the American people must care about fierce dependence on foreign government\xe2\x80\x99s spending.  In party, but he wouldn\xe2\x80\x99t see law, is health care -- because he believed it was exceptionally than in South Africa -- a willingness to achieve, an issue heavoncoty doesn\xe2\x80\x99t just award this law and respect elections, and that\xe2\x80\x99s the promise of insurance companies together and go achieved